In [1]:
#imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchWindowException,TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import time

In [2]:
def open_browser():
    global driver
    driver=webdriver.Chrome()

def get_website(websitename):
    try:
        driver.get(websitename)
    except NoSuchWindowException:
        print("Reopening window")
        open_browser()
        driver.get(websitename)  
def signin(phone):
    sign_in_ele=WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="id_a"] | //*[@id=":ra:"]/div'))
    )
    sign_in_ele.click()
    sign_in_personal=WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="b_idProviders"]/li[1]/a/span | //*[@id=":r9:"]/div/div[1]/button'))
    )
    sign_in_personal.click()
    user=WebDriverWait(driver,10).until(
    EC.element_to_be_clickable((By.XPATH,'//*[@id="i0116"]'))
    )
    user.send_keys(phone)
    user.send_keys(Keys.RETURN)
def say():
    try:
        say_no=WebDriverWait(driver,15).until(
    EC.element_to_be_clickable((By.XPATH,'//*[@id="declineButton"]'))
    )
        say_no.click()
    except TimeoutException:
        print("That Other UI man")
        send_notification=WebDriverWait(driver,10).until(
        EC.element_to_be_clickable((By.Xpath,'//*[@id="pageContent"]/div/form/div[4]/div/div/div/button'))
        )
        send_notification.click()
        say()

In [3]:
def message(query):
    ans = []
    messagebox=driver.find_element(By.XPATH,'//*[@id="userInput"]')
    
    
    for q in query:
        messagebox.send_keys(q)
        messagebox.send_keys(Keys.RETURN)
        chat_reply=WebDriverWait(driver,10).until(
        EC.element_to_be_clickable((By.XPATH,'//*[@id="app"]/main/div[3]/div/div/div[2]/div[2]/div[2]/div/p/span | //*[@id="app"]/main/div[3]/div/div/div[2]/div/div[2]/div/p/span'))
        )
        time.sleep(4)
        ans.append(chat_reply.text)
    
    return ans

In [4]:
def messagebox():
    parent1 = driver.find_element(By.CSS_SELECTOR, '#b_sydConvCont > cib-serp')
    s1 = driver.execute_script("return arguments[0].shadowRoot", parent1)
    parent2 = s1.find_element(By.CSS_SELECTOR, 'cib-action-bar')
    s2 = driver.execute_script("return arguments[0].shadowRoot", parent2)
    parent3 = s2.find_element(By.CSS_SELECTOR, 'div.main-container')
    search_box = parent3.find_element(By.CSS_SELECTOR, 'cib-text-input')
    s3 = driver.execute_script("return arguments[0].shadowRoot", search_box)
    global textbox
    textbox = s3.find_element(By.CSS_SELECTOR, '#searchbox')
    textbox.click()
    #textbox.send_keys("Who is the father of india")
    return textbox

In [6]:
def answerbox1():
    script = """
    const p1 = document.querySelector('#b_sydConvCont > cib-serp');
    const s1 = p1.shadowRoot;
    const p2 = s1.querySelector('#cib-conversation-main');
    const s2 = p2.shadowRoot;
    const p3 = s2.querySelector('#cib-chat-main');

    // Collect all shadow roots and their host elements from each message group inside each chat turn
    const shadowInfo = [];
    const chatTurns = p3.querySelectorAll('cib-chat-turn');

    chatTurns.forEach(turn => {
        const turnShadowRoot = turn.shadowRoot;
        if (turnShadowRoot) {
            const messageGroups = turnShadowRoot.querySelectorAll('cib-message-group.response-message-group');
            messageGroups.forEach(group => {
                const groupShadowRoot = group.shadowRoot;
                if (groupShadowRoot) {
                    // Store both the host element and its shadow root
                    shadowInfo.push({
                        host: group, // The host element (cib-message-group)
                        shadowRoot: groupShadowRoot
                    });
                }
            });
        }
    });

    // Return the array of shadow info (hosts and their shadow roots)
    return shadowInfo;
    """
    
    # Execute the script in the context of the current page
    shadow_info = driver.execute_script(script)

    # Provide debug information about the number of shadow roots found
    if not shadow_info or len(shadow_info) == 0:  # Use len() for Python
        print("No shadow roots found in the specified structure.")
    else:
        print(f"Found {len(shadow_info)} shadow roots and their hosts.")

    return shadow_info


In [21]:
def get_aria_label(shadow_info):
    # Check if there are any shadow roots
    if not shadow_info:
        print("No shadow roots found.")
        return None

    # Get the last shadow root and its corresponding host element
    last_info = shadow_info[-1]  # This should be a dictionary with 'host' and 'shadowRoot'
    last_host_element = last_info['host']

    # Use JavaScript to access the shadow root from the host element and query for the cib-message element
    script = """
    const hostElement = arguments[0];
    const shadowRoot = hostElement.shadowRoot;
    const messageElement = shadowRoot.querySelector('cib-message');
    return messageElement ? messageElement.getAttribute('aria-label') : null;
    """

    # Execute the script with the last host element
    aria_label = driver.execute_script(script, last_host_element)

    return aria_label


In [13]:
def reset_chat(driver):
    actions = ActionChains(driver)
    actions.key_down(Keys.CONTROL).send_keys('r').key_up(Keys.CONTROL).perform()
    time.sleep(3)
    shadow_info=answerbox(driver)
    return shadow_info

In [30]:
def main():
    global shadow_info
    global driver
    open_browser()
    get_website("https://copilot.microsoft.com/")
    signin(9886449568)
    say()
    question1 = "gandhi is the father of nation"
    paraphrased_question1="Indias father of nation is gandhi"
    comparision_query = 'Hey CoPiolot ill give you two questions can you give Me "Yes they are similar and By % they are similar" if they are similar if not then give "no they are not similar because and reason if they are not similar"'
    textb=messagebox()
    textb.send_keys(comparision_query)
    textb.send_keys(Keys.RETURN)
    time.sleep(4)
    textb.send_keys(question1)
    textb.send_keys(' "and" ')
    textb.send_keys(paraphrased_question1)
    textb.send_keys(Keys.RETURN)
    time.sleep(4)
    ansb=answerbox1()
    label = get_aria_label(ansb)
    return label

In [32]:
main()

Found 2 shadow roots and their hosts.


'Sent by Copilot: Yes, they are similar. By 100%, they are similar.'

In [31]:
driver.quit()

In [26]:
ansb=answerbox1()

Found 1 shadow roots and their hosts.


In [27]:
get_aria_label(ansb)

"Sent by Copilot: Sure, I'd be happy to help with that! Please go ahead and provide the two questions you'd like me to compare."